In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel("outputs/complete_df.xlsx", parse_dates=[1, 85])

In [ ]:
df

In [ ]:
miembros_familia = pd.read_excel(
    "inputs/Encuesta_para_inmigrantes_venezolanos_en_Ica_-_all_versions_-_labels_-_2023-02-18-20-40-30.xlsx",
    sheet_name=1,
)

In [ ]:
miembros_familia.head()

In [ ]:
miembros_familia_sexo = (
    miembros_familia.rename(columns={"Indique el sexo del miembro de familia": "sexo"})
    .groupby(by=["_parent_index", "sexo"], as_index=False)
    .size()
    .pivot(index="_parent_index", columns="sexo", values="size")
    .fillna(0)
)

In [ ]:
miembros_familia_sexo.columns = [
    f"n_miembros_{values.lower()}" for values in miembros_familia_sexo.columns
]

In [ ]:
miembros_familia_sexo

In [ ]:
miembros_familia = miembros_familia.rename(
    columns={"Indique la edad del miembro de familia": "edad"}
)
miembros_familia["grupo_edad"] = pd.cut(
    x=miembros_familia["edad"],
    bins=[0, 17, 64, 100],
    labels=["menores", "adultos", "mayores"],
)

In [ ]:
miembros_familia_edad = (
    miembros_familia.groupby(by=["_parent_index", "grupo_edad"], as_index=False)
    .size()
    .pivot(index="_parent_index", columns="grupo_edad", values="size")
    .fillna(0)
)
miembros_familia_edad.columns = [
    f"n_miembros_{values}" for values in miembros_familia_edad.columns
]

In [ ]:
miembros_familia_edad

In [ ]:
miembros_cols = pd.concat((miembros_familia_edad, miembros_familia_sexo), axis=1)
miembros_cols = miembros_cols.reset_index()
miembros_cols["_parent_index"] = miembros_cols["_parent_index"].astype(int)

In [ ]:
# Sanity check
miembros_cols.isna().sum()

In [ ]:
df = df.merge(
    right=miembros_cols, how="left", left_on="_index", right_on="_parent_index"
).drop(labels="_parent_index", axis=1)
df.loc[:, miembros_cols.columns[1:]] = df[miembros_cols.columns[1:]].fillna(0.0)

In [ ]:
df

In [ ]:
alimentos = pd.read_excel(
    "inputs/Encuesta_para_inmigrantes_venezolanos_en_Ica_-_all_versions_-_labels_-_2023-02-18-20-40-30.xlsx",
    sheet_name=2,
)

In [ ]:
alimentos.shape

In [ ]:
alimentos.head()

In [ ]:
alimentos["Seleccionar alimento"].unique()

In [ ]:
alimentos["Seleccionar alimento"].isna().sum()

In [ ]:
alimentos = alimentos.dropna(
    subset=["Seleccionar alimento", "Especifica otro alimento"], how="all"
)

In [ ]:
alimentos[alimentos["Seleccionar alimento"].isna()]

In [ ]:
alimentos[alimentos["Seleccionar alimento"] == "Otro, especificar"][
    "Especifica otro alimento"
].sort_values().unique().tolist()

In [ ]:
to_replace = {
    ## Opciones iniciales
    "Derivados lacteos": ["QUESO", "Queso", "queso"],
    "Pescado": [
        "ATUN",
        "Atún",
        "Atún más que la semana y menestras",
        "Atún y Sandina",
        "ENLATADOS",
    ],
    "Otros": [
        "Hojado 4 veces a la semana",
        "Hotdog",
        "Menudencias solo hacen sopita",
        "Sangresita",
    ],
    # No origen animal
    np.nan: [
        "Verduras",
        "GRANOS",
        "Grano",
        "Granos",
        "granos",
        "MENESTRAS",
        "Menestras",
        "Menestras come cuando los vecinos le dan",
        "Menestras: Lentejas, Alverjas, caraotas, etc.",
        "Menestras: lentejas, alberjas, caraota, etc.",
        "Menestras: lentejas, alverjas, caraotas, etc.",
        "Menestras: lentejas, alverjas, pallares, etc.",
        "Pasta",
        "Pastas",
        "Arroz y menestra",
        "Arroz, pasta, granos y arina",
        "Granos y menestras",
        "Granos, menestras",
        "Harinas, menestras, pastras",
        "Menestras, fideos y arroz",
    ],
}

In [ ]:
to_replace_dict = {e: k for k, el in to_replace.items() for e in el}

In [ ]:
alimentos["Especifica otro alimento"] = alimentos["Especifica otro alimento"].replace(
    to_replace_dict
)

In [ ]:
alimentos[alimentos["Seleccionar alimento"] == "Otro, especificar"][
    "Especifica otro alimento"
].sort_values().unique().tolist()

In [ ]:
alimentos.loc[
    (alimentos["Seleccionar alimento"] == "Otro, especificar")
    & (alimentos["Especifica otro alimento"].isna()),
    "Seleccionar alimento",
] = np.nan

In [ ]:
alimentos[alimentos["Seleccionar alimento"] == "Otro, especificar"][
    "Especifica otro alimento"
].sort_values().unique().tolist()

In [ ]:
alimentos.loc[
    (alimentos["Seleccionar alimento"] == "Otro, especificar")
    & (alimentos["Especifica otro alimento"].isna()),
    "Especifica otro alimento",
]

In [ ]:
alimentos["Seleccionar alimento"] = alimentos["Seleccionar alimento"].replace(
    {"Otro, especificar": np.nan}
)
alimentos["Seleccionar alimento"] = alimentos["Seleccionar alimento"].fillna(
    alimentos["Especifica otro alimento"]
)

In [ ]:
alimentos["Seleccionar alimento"].unique()

In [ ]:
alimentos[alimentos["Seleccionar alimento"].isna()][
    "Especifica otro alimento"
].isna().all()

In [ ]:
alimentos = alimentos.dropna(
    subset=["Seleccionar alimento", "Especifica otro alimento"], how="all"
)

In [ ]:
alimentos["Seleccionar alimento"].unique()

In [ ]:
alimentos_df = alimentos[
    [
        "Seleccionar alimento",
        "¿Cuántas veces a la semana consume cada alimento de orígen animal mencionado en la pregunta anterior?",
        "_parent_index",
    ]
]

In [ ]:
tipo_alimentos = (
    alimentos_df.rename(columns={"Seleccionar alimento": "tipo_alimento"})
    .groupby(by=["_parent_index", "tipo_alimento"], as_index=False)
    .size()
    .pivot(index="_parent_index", columns="tipo_alimento", values="size")
    .fillna(0)
)

In [ ]:
new_cols_order = tipo_alimentos.columns.drop("Otros").tolist()
new_cols_order.append("Otros")
tipo_alimentos = tipo_alimentos[new_cols_order]

In [ ]:
tipo_alimentos.columns = [
    f"consume_alimento_orig_animal_{values.lower()}"
    for values in tipo_alimentos.columns
]

In [ ]:
alimentos_df[
    "¿Cuántas veces a la semana consume cada alimento de orígen animal mencionado en la pregunta anterior?"
].describe()

In [ ]:
freq_consumo_alimentos = (
    alimentos_df.rename(
        columns={
            "Seleccionar alimento": "alimento",
            "¿Cuántas veces a la semana consume cada alimento de orígen animal mencionado en la pregunta anterior?": "freq_consumo",
        }
    )
    .groupby(by=["_parent_index", "alimento"], as_index=False)["freq_consumo"]
    .mean()
    .pivot(index="_parent_index", columns="alimento", values="freq_consumo")
    .fillna(0)
)

In [ ]:
freq_consumo_alimentos = freq_consumo_alimentos[new_cols_order]

In [ ]:
freq_consumo_alimentos.columns = [
    f"freq_consumo_{values.lower()}" for values in tipo_alimentos.columns
]

In [ ]:
freq_consumo_alimentos.shape

In [ ]:
tipo_alimentos.shape

In [ ]:
alimentos_cols = pd.concat((tipo_alimentos, freq_consumo_alimentos), axis=1)
alimentos_cols = alimentos_cols.reset_index()
alimentos_cols["_parent_index"] = alimentos_cols["_parent_index"].astype(int)

In [ ]:
# Sanity check
alimentos_cols.isna().sum()

In [ ]:
col_order = [
    "_parent_index",
    "consume_alimento_orig_animal_cerdo",
    "freq_consumo_consume_alimento_orig_animal_cerdo",
    "consume_alimento_orig_animal_derivados lacteos",
    "freq_consumo_consume_alimento_orig_animal_derivados lacteos",
    "consume_alimento_orig_animal_huevos",
    "freq_consumo_consume_alimento_orig_animal_huevos",
    "consume_alimento_orig_animal_leche",
    "freq_consumo_consume_alimento_orig_animal_leche",
    "consume_alimento_orig_animal_pescado",
    "freq_consumo_consume_alimento_orig_animal_pescado",
    "consume_alimento_orig_animal_pollo",
    "freq_consumo_consume_alimento_orig_animal_pollo",
    "consume_alimento_orig_animal_res",
    "freq_consumo_consume_alimento_orig_animal_res",
    "consume_alimento_orig_animal_otros",
    "freq_consumo_consume_alimento_orig_animal_otros",
]

In [ ]:
alimentos_cols = alimentos_cols[col_order]

In [ ]:
df = df.merge(
    right=alimentos_cols, how="left", left_on="_index", right_on="_parent_index"
).drop(labels="_parent_index", axis=1)
df.loc[:, alimentos_cols.columns[1:]] = df[alimentos_cols.columns[1:]].fillna(0.0)

In [ ]:
df.columns[df.columns.str.startswith("_")]

In [ ]:
df = df.drop(
    [
        "__version__",
        "_uuid",
        "_submission_time",
        "_validation_status",
        "_notes",
        "_status",
        "_submitted_by",
        "_tags",
        "_index",
        "new_id",
    ],
    axis=1,
)

In [ ]:
df.to_excel("encuesta_agregada.xlsx", index=False)

In [ ]:
## Codigo que considera opciones que no son de origen animal

# to_replace = {
#     ## Opciones iniciales
#     'Derivados lacteos': ['QUESO', 'Queso', 'queso'],

#     'Pescado': ['ATUN', 'Atún', 'Atún más que la semana y menestras',
#                 'Atún y Sandina', 'ENLATADOS'],

#     ## Nuevas opciones
#     'Granos': ['GRANOS', 'Grano', 'Granos', 'granos'],

#     'Menestras': ['MENESTRAS', 'Menestras', 'Menestras come cuando los vecinos le dan',
#                   'Menestras: Lentejas, Alverjas, caraotas, etc.',
#                   'Menestras: lentejas, alberjas, caraota, etc.',
#                   'Menestras: lentejas, alverjas, caraotas, etc.',
#                   'Menestras: lentejas, alverjas, pallares, etc.'],

#     'Pastas': ['Pasta', 'Pastas'],

#     'Otros': ['Hojado 4 veces a la semana', 'Hotdog', 'Menudencias solo hacen sopita', 'Sangresita'],

#     ## No valido
#     np.nan: ['Verduras']
# }


# to_replace_dict = {e: k for k, el in to_replace.items() for e in el}

# alimentos['Especifica otro alimento'] = alimentos['Especifica otro alimento'].replace(to_replace_dict)

# alimentos[alimentos["Seleccionar alimento"] == 'Otro, especificar']['Especifica otro alimento'].sort_values().unique().tolist()

# special_vals = ['Arroz y menestra',
# 'Arroz, pasta, granos y arina',
# 'Menestras, fideos y arroz',
# 'Granos y menestras',
# 'Granos, menestras',
# 'Harinas, menestras, pastras']

# otro_alimento = (alimentos[
#         alimentos['Especifica otro alimento'].isin(special_vals)
#     ]['Especifica otro alimento']
#     .str.lower()
#     .str.replace('y', ',')
#     .str.replace(" ", "")
#     .str.split(',', expand=True)
#     .replace({'menestra': 'menestras',
#               'pastras': 'pastas',
#               'pasta': 'pastas',
#               'fideos': 'pastas',
#               'arina': 'harinas'})
#     .reset_index()
#     .melt('index')
#     .drop('variable', axis=1)
#     .dropna()
# )

# otro_alimento['value'] = otro_alimento['value'].str.title()

# otro_alimento = (otro_alimento
#     .rename(columns={'value': 'Especifica otro alimento'})
#     .set_index('index')
#     .merge(alimentos[['¿Cuántas veces a la semana consume cada alimento de orígen animal mencionado en la pregunta anterior?',
#                       '_parent_index']], left_index=True, right_index=True, how='left')
# )

# alimentos.shape

# len(otro_alimento.index.unique())

# # Drop special "other" values
# alimentos[~alimentos.index.isin(otro_alimento.index.unique())]

# alimentos = alimentos[~alimentos.index.isin(otro_alimento.index.unique())]

# completo_alimentos = pd.concat((alimentos, otro_alimento), ignore_index=True)

# completo_alimentos[completo_alimentos["Seleccionar alimento"] == 'Otro, especificar']['Especifica otro alimento'].unique().tolist()

# completo_alimentos[completo_alimentos["Seleccionar alimento"] == 'Otro, especificar']['Especifica otro alimento'].isna().sum()

# otro_alimento.index.unique()

# completo_alimentos[(completo_alimentos["Seleccionar alimento"] == 'Otro, especificar') & (completo_alimentos["Especifica otro alimento"].isna())]

# completo_alimentos[(completo_alimentos["Seleccionar alimento"] == 'Otro, especificar') & (completo_alimentos["Especifica otro alimento"].isna())] = 'Otros'

# completo_alimentos["Seleccionar alimento"] = completo_alimentos["Seleccionar alimento"].replace({'Otro, especificar': np.nan})

# completo_alimentos["Seleccionar alimento"] = completo_alimentos["Seleccionar alimento"].fillna(completo_alimentos["Especifica otro alimento"])

# tipo_alimentos = (completo_alimentos
#     .rename(columns={'Seleccionar alimento': 'alimento'})
#     .groupby(by=['_parent_index', 'alimento'], as_index=False)
#     .size()
#     .pivot(index='_parent_index', columns='alimento', values="size")
#     .fillna(0))

# tipo_alimentos.columns

# tipo_alimentos.columns = ['Arroz', 'Cerdo', 'Derivados lacteos', 'Granos', 'Harinas', 'Huevos',
#                           'Leche', 'Menestras', 'Pastas', 'Pescado', 'Pollo', 'Res', 'Otros']

# tipo_alimentos.columns = [f"consume_alimento_orig_animal_{values.lower()}"
#                                  for values in tipo_alimentos.columns]

# tipo_alimentos